In [1]:
import pandas as pd
import numpy as np
import json
import datetime

In [2]:
forecast_date_str="2018-07-10" # Tuesday
inclusion_store_date="2018_07_07" # Last Saturday
folder="/home/jian/Projects/Big_Lots/Weather/Json_data/forecast/"
json_forecast=json.load(open(folder+forecast_date_str+".json","r"))
list_k=(5-datetime.datetime.strptime(forecast_date_str,"%Y-%m-%d").date().weekday())*8-1
inclusion_store_df=pd.read_table("/home/jian/BigLots/MediaStorm_"+inclusion_store_date+"/MediaStormSalesWeekly.txt",dtype=str,sep="|")
inclusion_store_df=inclusion_store_df[~inclusion_store_df['location_id'].isin(['145','6990'])]

In [3]:
inclusion_store_df['subclass_gross_sales_amt']=inclusion_store_df['subclass_gross_sales_amt'].astype(float)
inclusion_store_df['gross_transaction_cnt']=inclusion_store_df['gross_transaction_cnt'].astype(float)

inclusion_store_df_trans=inclusion_store_df[['location_id','week_end_dt','gross_transaction_cnt']].drop_duplicates()
inclusion_store_df_sales=inclusion_store_df.groupby(['location_id','week_end_dt'])['subclass_gross_sales_amt'].sum().reset_index()

inclusion_store_df_trans=inclusion_store_df_trans.rename(columns={"gross_transaction_cnt":"transaction"})
inclusion_store_df_sales=inclusion_store_df_sales.rename(columns={"subclass_gross_sales_amt":"sales"})
df_stores=pd.merge(inclusion_store_df_sales,inclusion_store_df_trans,on=["location_id","week_end_dt"],how="outer")

In [4]:
store_list_txt=pd.read_table("/home/jian/BigLots/static_files/MediaStormStores_20180703.txt",sep="|",dtype=str)
store_list_txt['zip_cd']=store_list_txt['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list_txt=store_list_txt[['location_id','location_desc','address_line_1','address_line_2','city_nm','state_nm','zip_cd']]

df_stores=pd.merge(df_stores,store_list_txt,on="location_id",how="left")
df_stores=df_stores.reset_index()
del df_stores['index']

zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,dtype=str)
zip_DMA=zip_DMA.iloc[:,[0,2]].drop_duplicates()
zip_DMA=zip_DMA.rename(columns={"CODE":"zip_cd","NAME":"DMA"})
zip_DMA=zip_DMA.drop_duplicates(['zip_cd'])
df_stores=pd.merge(df_stores,zip_DMA,on="zip_cd",how="left")

In [5]:
df_stores.shape

(1409, 11)

In [6]:
rain_snow_weight=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Q1_inclusion_store_all_weather_type_ranked.xlsx",sheetname="ranked rain snow")

rain_weight=rain_snow_weight.iloc[:,[4,6]]
rain_weight.columns=[['rain_desc','weight']]
rain_weight['weight']=rain_weight['weight'].astype(int)
rain_weight=rain_weight.sort_values("weight")

snow_weight=rain_snow_weight.iloc[:,[0,2]]
snow_weight.columns=[['snow_desc','weight']]
snow_weight=snow_weight[~pd.isnull(snow_weight['snow_desc'])]
snow_weight['weight']=snow_weight['weight'].astype(int)
snow_weight=snow_weight.sort_values("weight")

group_weight=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Q1_inclusion_store_all_weather_type_ranked.xlsx",sheetname="all_weather_group_list")
group_weight['Severity']=group_weight['Severity'].astype(int)
group_weight=group_weight[['all_type_group','Severity']]
group_weight_dict=group_weight[['all_type_group', 'Severity']].set_index('all_type_group').T.to_dict()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [7]:
output_forecast_weather=pd.DataFrame()
index_num=0
for zip_cd in df_stores['zip_cd'].unique().tolist():
    if zip_cd in list(json_forecast.keys()):
        weather_list=json_forecast[zip_cd]['list'][list_k]['weather']
        forecast_time=datetime.datetime.fromtimestamp(json_forecast[zip_cd]['list'][list_k]['dt'])
        all_forecast_group_value_zip=[]
        all_forecast_desc_value_zip=[]
        for j in range(len(weather_list)):
            weather_forecast_group=weather_list[j]['main']
            weather_forecast_desc=weather_list[j]['description']
            all_forecast_group_value_zip=list(set(all_forecast_group_value_zip+[weather_forecast_group]))
            all_forecast_desc_value_zip=list(set(all_forecast_desc_value_zip+[weather_forecast_group]))

        all_forecast_group_severity_zip=[]
        all_forecast_desc_severity_zip=[]
        for k in range(len(all_forecast_group_value_zip)):
            forecast_severity_zip=group_weight_dict[all_forecast_group_value_zip[k]]['Severity']
            all_forecast_group_severity_zip=list(set(all_forecast_group_severity_zip+[forecast_severity_zip]))
            all_forecast_desc_severity_zip
            if k==0:
                selected_havest_forecast_group_value_zip=all_forecast_group_value_zip[0]
                # selected_havest_forecast_desc_value_zip=
            else:
                if group_weight_dict[all_forecast_group_value_zip[k]]['Severity']>group_weight_dict[selected_havest_forecast_group_value_zip]['Severity']:
                    selected_havest_forecast_group_value_zip=all_forecast_group_value_zip[k]
                    # selected_havest_forecast_desc_value_zip
        forecast_max_severity_zip=max(all_forecast_group_severity_zip)

        df_app=pd.DataFrame({"zip_cd":zip_cd,"Forecast_Time":forecast_time,"Forecast_Severity":forecast_max_severity_zip,
                            "Forecast_Weather_Type":selected_havest_forecast_group_value_zip},index=[index_num])
        index_num=index_num+1
        output_forecast_weather=output_forecast_weather.append(df_app)
    else:
        print(zip_cd,"Not Collected")
        df_app=pd.DataFrame({"zip_cd":zip_cd,"Forecast_Time":"Not_Collected","Forecast_Severity":'Not_Collected',
                            "Forecast_Weather_Type":'Not_Collected'},index=[index_num])
        index_num=index_num+1
        output_forecast_weather=output_forecast_weather.append(df_app)

14304 Not Collected


In [8]:
output=pd.merge(df_stores,output_forecast_weather,on="zip_cd",how="left")
week_end_date=output['week_end_dt'].unique()[0]
Saturday_str=str(output[output['Forecast_Time']!='Not_Collected']['Forecast_Time'].apply(lambda x: x.date()).unique()[0])
output['location_id']=output['location_id'].astype(int)
output=output.sort_values('location_id')

In [9]:
output.to_csv("/home/jian/BiglotsCode/outputs/Output_"+week_end_date+"/weather_forecast_for_Saturday_"+Saturday_str+".csv",index=False)